In [1]:
# download libraries
# ----------
!pip install spacy
!python -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 245.8 kB/s eta 0:00:52
     --------------------------------------- 0.2/12.8 MB 833.5 kB/s eta 0:00:16
     -- ------------------------------------- 0.8/12.8 MB 3.8 MB/s eta 0:00:04
     ------ --------------------------------- 1.9/12.8 MB 7.3 MB/s eta 0:00:02
     --------- ------------------------------ 3.0/12.8 MB 9.6 MB/s eta 0:00:02
     ------------ --------------------------- 4.0/12.8 MB 11.7 MB/s eta 0:00:01
     ---------------- ----------------------- 5.2/12.8 MB 13.1 MB/s eta 0:00:01
     ------------------- -------------------- 6.3/12.8 MB 14.4 MB/s eta

In [10]:
import os
os.getcwd()

'C:\\Users\\OWNER\\SOCIOL273M'

In [2]:
# load libraries
# ----------
import spacy
import en_core_web_sm
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sn

# settings
%matplotlib inline

In [15]:
cfpb = pd.read_csv("C:/Users/OWNER/SOCIOL273M/data/CFPB 2020 Complaints.csv")

In [3]:
# load the data
# ----------

# load the dataframe
cfpb = pd.read_csv("D:/HYEM'S/PH.D/Classes/2nd (24.spring)/SOCIOL 273M/handouts/data/CFPB 2020 Complaints.csv")

#a = pd.read_csv("../../data/CFPB 2020 Complaints.csv")

# drop missing on "Consumer complaint narrative" feature and reset the index bc we've dropped
cfpb = cfpb.dropna(subset = ['Consumer complaint narrative']).reset_index()

In [4]:
# view 
# ----------
cfpb.head()

,index,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,8,01/30/20,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Investigation took more than 30 days,Reviewed my credit report in XX/XX/XXXX and no...,NaN,"EQUIFAX, INC.",AZ,850XX,NaN,Consent provided,Web,01/30/20,Closed with explanation,Yes,NaN,3515096
1,10,03/12/20,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Account status incorrect,TransUnion has not properly investigated the i...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",IL,606XX,NaN,Consent provided,Web,03/12/20,Closed with explanation,Yes,NaN,3564439
2,14,05/01/20,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,XX/XX/2020 someone tried to steal my identity ...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",IL,606XX,NaN,Consent provided,Web,05/01/20,Closed with non-monetary relief,Yes,NaN,3633318
3,19,04/06/20,Debt collection,Other debt,Attempts to collect debt not owed,Debt was paid,I paid the debt on XX/XX/XXXX. I disputed acco...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,NY,NaN,Servicemember,Consent provided,Web,04/06/20,Closed with explanation,Yes,NaN,3594679
4,35,04/18/20,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,A COLLECTION HAS BEEN REPORTED TO MY CREDIT RE...,Company has responded to the consumer and the ...,"Convergent Resources, Inc.",FL,336XX,NaN,Consent provided,Web,04/18/20,Closed with explanation,Yes,NaN,3611900


In [22]:
paragraph

'Reviewed my credit report in XX/XX/XXXX and noticed a lot of errors, inconsistent, and incorrect information. Sent a letter to Equifax on XX/XX/XXXX via mail asking them for an investigation and to verify all the dates and amounts were correct and fix the incorrect reporting on my credit. They did not respond at all so I sent another letter on XX/XX/XXXX via mail, again asking for an investigation and proof. They still didnt respond to that letter so I sent a third letter on XX/XX/XXXX certified mail so I have proof that they signed for my letter.\n\nLast week I received two letters from Equifax dated XX/XX/XXXX on the same day. The said that they could not locate my credit file and needed me to send proof of identification and address. With all three letters I sent a copy of my Arizona drivers license and my XXXX direct deposit sub as my proof of address. The second letter said that they received my request to be removed from the promotions list and that it was added to my credit fil

In [16]:
text = cfpb['Consumer complaint narrative'][0:100]
paragraph = ''.join(text)
sentences_list = paragraph.split(".")
for s in sentences_list:                            # take just the first 5 sentences
    print(s + '\n')                             # "\" sets it on a new line

Reviewed my credit report in XX/XX/XXXX and noticed a lot of errors, inconsistent, and incorrect information

 Sent a letter to Equifax on XX/XX/XXXX via mail asking them for an investigation and to verify all the dates and amounts were correct and fix the incorrect reporting on my credit

 They did not respond at all so I sent another letter on XX/XX/XXXX via mail, again asking for an investigation and proof

 They still didnt respond to that letter so I sent a third letter on XX/XX/XXXX certified mail so I have proof that they signed for my letter



Last week I received two letters from Equifax dated XX/XX/XXXX on the same day

 The said that they could not locate my credit file and needed me to send proof of identification and address

 With all three letters I sent a copy of my Arizona drivers license and my XXXX direct deposit sub as my proof of address

 The second letter said that they received my request to be removed from the promotions list and that it was added to my credit

In [17]:
# use a different parameter in the split function to create tokens using a " " (space)
# ----------
sentence = sentences_list                 # take the first sentence from our "sentences" object
print("Sentence to split: ", sentence)  # visualize the sentence we we splitting
tokens = [item.split() for item in sentence]
tokens                                  # print the token

Sentence to split:  ['Reviewed my credit report in XX/XX/XXXX and noticed a lot of errors, inconsistent, and incorrect information', ' Sent a letter to Equifax on XX/XX/XXXX via mail asking them for an investigation and to verify all the dates and amounts were correct and fix the incorrect reporting on my credit', ' They did not respond at all so I sent another letter on XX/XX/XXXX via mail, again asking for an investigation and proof', ' They still didnt respond to that letter so I sent a third letter on XX/XX/XXXX certified mail so I have proof that they signed for my letter', '\n\nLast week I received two letters from Equifax dated XX/XX/XXXX on the same day', ' The said that they could not locate my credit file and needed me to send proof of identification and address', ' With all three letters I sent a copy of my Arizona drivers license and my XXXX direct deposit sub as my proof of address', ' The second letter said that they received my request to be removed from the promotions l

[['Reviewed',
  'my',
  'credit',
  'report',
  'in',
  'XX/XX/XXXX',
  'and',
  'noticed',
  'a',
  'lot',
  'of',
  'errors,',
  'inconsistent,',
  'and',
  'incorrect',
  'information'],
 ['Sent',
  'a',
  'letter',
  'to',
  'Equifax',
  'on',
  'XX/XX/XXXX',
  'via',
  'mail',
  'asking',
  'them',
  'for',
  'an',
  'investigation',
  'and',
  'to',
  'verify',
  'all',
  'the',
  'dates',
  'and',
  'amounts',
  'were',
  'correct',
  'and',
  'fix',
  'the',
  'incorrect',
  'reporting',
  'on',
  'my',
  'credit'],
 ['They',
  'did',
  'not',
  'respond',
  'at',
  'all',
  'so',
  'I',
  'sent',
  'another',
  'letter',
  'on',
  'XX/XX/XXXX',
  'via',
  'mail,',
  'again',
  'asking',
  'for',
  'an',
  'investigation',
  'and',
  'proof'],
 ['They',
  'still',
  'didnt',
  'respond',
  'to',
  'that',
  'letter',
  'so',
  'I',
  'sent',
  'a',
  'third',
  'letter',
  'on',
  'XX/XX/XXXX',
  'certified',
  'mail',
  'so',
  'I',
  'have',
  'proof',
  'that',
  'they',
  '

In [23]:
# use a different parameter in the split function to create tokens using a " " (space)
# ----------
nlp = en_core_web_sm.load()                 # load the small language model en_core_web_sm

doc = nlp(paragraph)                             # apply the nlp language model to process the text object from above
spacy_words = [token.text for token in doc] # create a list of words from doc object (this is a list comprehension)
display(f"Tokenized words: {spacy_words}")  # fancier way to displaying an object using the f-string formatting
#print(spacy_words)                         # another way that you might be more familier with

# note that "token.text" is an attribute of "token" not the object text we are cleaning
# see--https://spacy.io/api/token--for more details

# general syntax for list comprehension: [expression for item in iterable]

'Tokenized words: [\'Reviewed\', \'my\', \'credit\', \'report\', \'in\', \'XX\', \'/\', \'XX\', \'/\', \'XXXX\', \'and\', \'noticed\', \'a\', \'lot\', \'of\', \'errors\', \',\', \'inconsistent\', \',\', \'and\', \'incorrect\', \'information\', \'.\', \'Sent\', \'a\', \'letter\', \'to\', \'Equifax\', \'on\', \'XX\', \'/\', \'XX\', \'/\', \'XXXX\', \'via\', \'mail\', \'asking\', \'them\', \'for\', \'an\', \'investigation\', \'and\', \'to\', \'verify\', \'all\', \'the\', \'dates\', \'and\', \'amounts\', \'were\', \'correct\', \'and\', \'fix\', \'the\', \'incorrect\', \'reporting\', \'on\', \'my\', \'credit\', \'.\', \'They\', \'did\', \'not\', \'respond\', \'at\', \'all\', \'so\', \'I\', \'sent\', \'another\', \'letter\', \'on\', \'XX\', \'/\', \'XX\', \'/\', \'XXXX\', \'via\', \'mail\', \',\', \'again\', \'asking\', \'for\', \'an\', \'investigation\', \'and\', \'proof\', \'.\', \'They\', \'still\', \'did\', \'nt\', \'respond\', \'to\', \'that\', \'letter\', \'so\', \'I\', \'sent\', \'a\'

In [24]:
# load STOP_WORDS module from spaCy library
# ----------
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation  # note that this is a base library

In [25]:
# load punctuation module from string library
# ----------

# initalize an empty list
tokens_reduced = []

# create a function to remove punctuation and stop words 
def rem_punc_stop(text):

    # set objects
    stop_words = STOP_WORDS         # set STOP_WORDS to a new object variable
    punc = set(punctuation)         # convert punctuation to a set
    
    # essentially remove the punctuation - important to remove punctuation first to correctly capture stop words
    punc_free = "".join([ch for ch in text if ch not in punc]) # join new list of characters (ch) in text w/ condition
                                                               # if ch is not in punctuation 
                                                               # "".join() creates a string from the list comprehension

    # apply nlp to punctuation-free object
    doc = nlp(punc_free)
    
    # extract words from processed text 
    spacy_words = [token.text for token in doc]
    
    # filter out words 
    no_punc = [word for word in spacy_words if word not in stop_words]
    
    # return
    return no_punc

In [26]:
# apply function
# ----------
# apply our new function to our text object from above
tokens_reduced = rem_punc_stop(paragraph)

In [27]:
# apply function to text object
# ----------
for entity in nlp(paragraph).ents:
    print(entity.text + ' - ' + entity.label_ + ' - ' + str(spacy.explain(entity.label_)))

XX - GPE - Countries, cities, states
Equifax - ORG - Companies, agencies, institutions, etc.
XX - GPE - Countries, cities, states
XX - GPE - Countries, cities, states
third - ORDINAL - "first", "second", etc.
XX - GPE - Countries, cities, states
Last week - DATE - Absolute or relative dates or periods
two - CARDINAL - Numerals that do not fall under another type
Equifax - ORG - Companies, agencies, institutions, etc.
XX - GPE - Countries, cities, states
the same day - DATE - Absolute or relative dates or periods
three - CARDINAL - Numerals that do not fall under another type
Arizona - GPE - Countries, cities, states
XXXX - PRODUCT - Objects, vehicles, foods, etc. (not services)
second - ORDINAL - "first", "second", etc.
XXXX XXXX XXXX XXXX - PERSON - People, including fictional
XXXX XXXX - PERSON - People, including fictional
XXXX XXXX - PERSON - People, including fictional
XXXX - PERSON - People, including fictional
XXXX XX - PERSON - People, including fictional
XXXX XXXX - PERSON - P

In [28]:
# now run a loop to identify the shortest variant of each word  from 
# ----------

lemma = [word.lemma_ for word in nlp(paragraph)]
lemmaa = ' '.join(lemma)

In [29]:
lemmaa

'review my credit report in XX / XX / XXXX and notice a lot of error , inconsistent , and incorrect information . send a letter to Equifax on XX / XX / xxxx via mail ask they for an investigation and to verify all the date and amount be correct and fix the incorrect reporting on my credit . they do not respond at all so I send another letter on XX / XX / xxxx via mail , again ask for an investigation and proof . they still do not respond to that letter so I send a third letter on XX / XX / XXXX certify mail so I have proof that they sign for my letter . \n\n last week I receive two letter from Equifax date XX / XX / XXXX on the same day . the say that they could not locate my credit file and need I to send proof of identification and address . with all three letter I send a copy of my Arizona driver license and my xxxx direct deposit sub as my proof of address . the second letter say that they receive my request to be remove from the promotion list and that it be add to my credit file 

In [30]:
tokens_reduced_again = rem_punc_stop(lemmaa)
tokens_reduced_again_text = ' '.join(tokens_reduced_again)
tokens_reduced_again_text

'review credit report XX   XX   XXXX notice lot error   inconsistent   incorrect information   send letter Equifax XX   XX   xxxx mail ask investigation verify date correct fix incorrect reporting credit   respond I send letter XX   XX   xxxx mail   ask investigation proof   respond letter I send letter XX   XX   XXXX certify mail I proof sign letter  \n\n  week I receive letter Equifax date XX   XX   XXXX day   locate credit file need I send proof identification address   letter I send copy Arizona driver license xxxx direct deposit sub proof address   second letter receive request remove promotion list add credit file   find file add restriction find credit file regard investigation purposestransunion properly investigate item   I request remove report pende thorough review  \n\n  account xxxx xxxx xxxx xxxx xxxx xxxx xxxx xx   XX2018 unauthorized xxxx xxxx XX   xx2018 unauthorized xxxx    xxxx XX   XX2018 unauthorized xxxx xxxx xxxx xx   xx2018 unauthorized xxxx    xxxx I xx   xx201

In [32]:
# chunking
# ----------
# apply the nlp to our text object to convert to a spacy object
doc = nlp(paragraph)

# run a loop to chunk and print the obejct
for np in doc.noun_chunks:
    print(np.text)

my credit report
XX/XX/XXXX
a lot
errors
inconsistent
incorrect information
a letter
Equifax
XX/XX/XXXX
mail
them
an investigation
all the dates
amounts
the incorrect reporting
my credit
They
I
another letter
XX/XX/XXXX
mail
an investigation
proof
They
that letter
I
a third letter
XX/XX/XXXX certified mail
I
proof
they
my letter
I
two letters
Equifax
dated XX/XX/XXXX
the same day
The
they
my credit file
me
proof
identification
address
all three letters
I
a copy
my Arizona drivers license
my XXXX direct deposit sub
my proof
address
The second letter
they
my request
the promotions list
it
my credit file
they
my file
the restriction
they
my credit file
regards
investigation purposes?TransUnion
the items
I
they
my report
a thorough review
XXXX XXXX XXXX XXXX
XXXX XXXX XXXX
I
not mineXX/XX/2020 someone
my identity
a credit card
XXXX  XXXX XXXX
XXXX XXXX XXXX XXXX
a hard inquiry XXXX
XXXX XXXX
my credit report
Transunion
XXXX
the XXXX XXXX hard inquiry
I
a letter
XXXX XXXX
Transunion
XXXX
th

In [33]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [34]:
# initalize CountVectorizer and specify some parameters
# ----------
bow_vector = CountVectorizer(tokenizer = rem_punc_stop, # use our function for tokenizing created above
                             token_pattern = None,      # set to "None" since we have specify our own pattern
                             ngram_range=(1,3))         # use default for unigrams - see documentation

In [35]:
tokens_reduced_again_text_a = [tokens_reduced_again_text]
cv = CountVectorizer()
cv.fit(tokens_reduced_again_text_a)

bow_matrix = bow_vector.fit_transform(tokens_reduced_again_text_a) 

In [36]:
bow_matrix.todense()

matrix([[1, 1, 1, ..., 1, 1, 1]], dtype=int64)